In [34]:
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import copy
import ast
import attributes

from ipywidgets import interact, Dropdown
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

pd.set_option('display.max_rows', 200)

In [35]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"
SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie.xlsx"

TypeError: 'str' object is not callable

In [36]:
xl = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, None);
df_table = xl['Objecten']
df_table.head(30)

,Unnamed: 0,Object,Tabellen,Overerven,Samenvoegen,ABR-code
0,0,Ignore,"["".*backup.*"", "".*kopie.*""]",NaN,NaN,NaN
1,1,Tekening,"[""^TEKENING.*""]",NaN,NaN,NaN
2,2,Vondst,"[""^VONDSTENLIJST"", ""^VONDST$""]",NaN,NaN,NaN
3,3,Spoor,"[""^SPOREN$"", ""^SPOOR$""]",NaN,NaN,NaN
4,4,Vulling,"[""^VULLING.*"", ""MUUR""]",NaN,NaN,NaN
5,5,Dia,"[""DIA.*""]",NaN,NaN,NaN
6,6,Foto,"[""FOTO.*""]",NaN,NaN,NaN
7,7,Put,"[""^PUT$"", ""PUTTEN""]",NaN,NaN,NaN
8,8,Artefact,"[""ARTEFACT.*""]",NaN,NaN,NaN
9,9,Hout,"[""HOUT"", ""ARTF_OPH""]",Artefact,NaN,NaN


## Find all Attributes and connect them to the Objects

In [37]:

df = attributes.getAllAttributes()
df[df['Object'] == 'Tekening']

,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld
1524,Tekening,(none),[],9,[],,NaN,NaN
1525,Tekening,ARCHIEFNR,[archiefnummer],7,"[DC024, None]",,NaN,NaN
1526,Tekening,CODE,"[schaal, code van de opgraving]",3,[],,NaN,NaN
1527,Tekening,CODENAAM,[(afgekorte) naam van de opgraving],1,[],,NaN,NaN
1528,Tekening,COUPE,[coupe tekening],10,"[DC024, None]",,NaN,NaN
1529,Tekening,DATUM,"[datum, datum tekenen]",11,"[DC024, None]",,NaN,NaN
1530,Tekening,DETAILS,"[detailtekening, details]",8,"[DC024, None]",,NaN,NaN
1531,Tekening,GEBRUIKER,[],6,[],,NaN,NaN
1532,Tekening,GEINKT,[geinkt],1,[],,NaN,NaN
1533,Tekening,GEWIJZIGD,[],6,[],,NaN,NaN


In [38]:
lst = df['Object'].unique()
Objecten_lst = Dropdown(options = sorted(lst))

@interact(obj = Objecten_lst)
def print_city(obj):
    display(df[df.Object == obj].style.apply(lambda x: ["color: green" if bool(set(x.projecten) & set(['DC179', 'DC154'])) else "" for v in x], axis = 1))


interactive(children=(Dropdown(description='obj', options=('Aardewerk', 'Artefact', 'Bot', 'Dia', 'Foto', 'Fot…

In [39]:
def setNr(kolom):
    string = str(kolom)
    return string[:len(string)-6] + 'nr' if string.endswith('nummer') else string

df["sugg_kolom"] = df["Kolommen"].str.lower()
#df["sugg_kolom"] = df.apply(lambda x: setNr(df["sugg_kolom"]), axis=1)
df["sugg_omschr"] = df.apply(lambda x: x['omschrijvingen'][0].split()[0].lower() if len(x['omschrijvingen']) > 0 else "", axis=1)
df["sugg_omschr"] = df.apply(lambda x: setNr(x['sugg_omschr']), axis=1)
df["new_attr"] = ""
df["new_attr_artefact"] = ""
df.head(30)

TypeError: 'str' object is not callable

In [40]:
lst_objects = df['Object'].unique()

with pd.ExcelWriter(SUGGESTIE_XL) as writer:
    for obj in lst_objects:
        df[df.Object == obj].to_excel(writer, sheet_name=obj)
    writer.save()


In [41]:
lst = list(df_table[df_table.Overerven == 'Artefact']['Object'])
lst.sort()
#lst = lst.sort()
for tp in lst:
    print(tp + ' = \"' + tp + '\"')

Aardewerk = "Aardewerk"
Bot = "Bot"
Glas = "Glas"
Hoorn = "Hoorn"
Hout = "Hout"
Ivoor = "Ivoor"
Keramiek = "Keramiek"
Kleipijp = "Kleipijp"
Leer = "Leer"
Menselijk_Materiaal = "Menselijk_Materiaal"
Metaal = "Metaal"
Munt = "Munt"
Onbekend = "Onbekend"
Schelp = "Schelp"
Spijker = "Spijker"
Steen = "Steen"
Textiel = "Textiel"


In [42]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)

with engine.connect() as connection:
    df = pd.read_sql_query('SELECT brondata, artefactsoort from "Def_Artefact"', connection)

list(df['artefactsoort'].unique())

['Aardewerk',
 'Glas',
 'Menselijk_Bot',
 'Dierlijk_Bot',
 'Hout',
 'Metaal',
 'Kleipijp',
 'Leer',
 'Steen']

In [43]:
lst

['Aardewerk',
 'Bot',
 'Glas',
 'Hoorn',
 'Hout',
 'Ivoor',
 'Keramiek',
 'Kleipijp',
 'Leer',
 'Menselijk_Materiaal',
 'Metaal',
 'Munt',
 'Onbekend',
 'Schelp',
 'Spijker',
 'Steen',
 'Textiel']

TypeError: 'str' object is not callable